# **A08 - Bootstrapping**
#### **Miguel Aaron Castillon Ochoa**
#### **Expediente:** 751858
#### **Fecha:** 20 / 11 / 2025

# **Configuracion, Carga y Preparacion de datos**

In [26]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import KFold
from statsmodels.api import OLS, add_constant, Logit

#  Carga de Datos 
df_adv = pd.read_csv("Advertising.csv", index_col=0) 
df_def = pd.read_csv("Default.csv")

#  1. Preparación para Regresión Lineal Múltiple (Advertising) 
X_adv_multi = df_adv[['TV', 'radio', 'newspaper']]
y_adv = df_adv['sales']
adv_cols = ['Intercepto', 'TV', 'radio', 'newspaper'] 

#  2. Preparación para Regresión Logística Múltiple (Default) 

df_def['default_num'] = df_def['default'].map({'No': 0, 'Yes': 1})


df_def_encoded = pd.get_dummies(df_def, columns=['student'], drop_first=True)
df_def_encoded.rename(columns={'student_Yes': 'student'}, inplace=True) 


X_def_multi = df_def_encoded[['balance', 'income', 'student']]
y_def = df_def_encoded['default_num']
def_cols = ['Intercepto', 'balance', 'income', 'student'] 

print("Preparación de datos completa.")

Preparación de datos completa.


# **Regresion Lineal Multiple (Dataset Advertising)**
## **Cálculo del Error Estándar (SE) por el Método Clásico**

In [27]:
# Ajuste del Modelo Clásico (OLS)
X_adv_sm = add_constant(X_adv_multi)
model_ols = OLS(y_adv, X_adv_sm).fit()

results_adv_classic = pd.DataFrame({
    'Coeficiente Clásico': model_ols.params,
    'SE Clásico': model_ols.bse
})

print("\nAdvertising: Resultados Clásicos (OLS)")
print(results_adv_classic)


Advertising: Resultados Clásicos (OLS)
           Coeficiente Clásico  SE Clásico
const                 2.938889    0.311908
TV                    0.045765    0.001395
radio                 0.188530    0.008611
newspaper            -0.001037    0.005871


## **Cálculo de Media y Desviación Estándar (SE) usando Bootstrap**
El bucle de Bootstrap realiza 1000 remuestreos y ajusta el modelo de regresión lineal en cada uno.

In [28]:
N_bootstrap = 1000
bootstrap_coeffs_adv = []

print("\nAdvertising: Cálculo de SE con Bootstrap")
for i in range(N_bootstrap):
    # Remuestreo con reemplazo (sample)
    sample_adv = df_adv.sample(n=len(df_adv), replace=True)
    
    # Ajuste del modelo
    X_boot = sample_adv[['TV', 'radio', 'newspaper']]
    y_boot = sample_adv['sales']
    model = LinearRegression().fit(X_boot, y_boot)
    
    # Guardar coeficientes: [Intercepto, TV, radio, newspaper]
    bootstrap_coeffs_adv.append([model.intercept_] + list(model.coef_))

# Calcular media y desviación estándar (SE Bootstrap)
coeffs_adv_array = np.array(bootstrap_coeffs_adv)
mean_bootstrap_adv = pd.Series(coeffs_adv_array.mean(axis=0), index=adv_cols).rename('Media Bootstrap')
se_bootstrap_adv = pd.Series(coeffs_adv_array.std(axis=0), index=adv_cols).rename('SE Bootstrap')

print("Resultados de los Coeficientes (Comparación Clásico vs. Bootstrap):")
print(pd.concat([results_adv_classic, mean_bootstrap_adv, se_bootstrap_adv], axis=1))


Advertising: Cálculo de SE con Bootstrap
Resultados de los Coeficientes (Comparación Clásico vs. Bootstrap):
            Coeficiente Clásico  SE Clásico  Media Bootstrap  SE Bootstrap
const                  2.938889    0.311908              NaN           NaN
TV                     0.045765    0.001395         0.045584      0.001910
radio                  0.188530    0.008611         0.189171      0.010684
newspaper             -0.001037    0.005871        -0.001138      0.006589
Intercepto                  NaN         NaN         2.957807      0.335080


# **Regresion Logistica Multiple (Dataset Default)**
## **Cálculo del Error Estándar (SE) por el Método Clásico**

In [30]:
X_def_sm = add_constant(X_def_multi).astype(float)
y_def_float = y_def.astype(float)

# Ajuste del Modelo Logit Clásico (Statsmodels)
model_logit = Logit(y_def_float, X_def_sm).fit(disp=0)

results_def_classic = pd.DataFrame({
    'Coeficiente Clásico': model_logit.params,
    'SE Clásico': model_logit.bse
})
results_def_classic.index = def_cols 

print("\nDefault: Resultados Clásicos (MLE)")
print(results_def_classic)


Default: Resultados Clásicos (MLE)
            Coeficiente Clásico  SE Clásico
Intercepto           -10.869045    0.492273
balance                0.005737    0.000232
income                 0.000003    0.000008
student               -0.646776    0.236257


## **Cálculo de Media y Desviación Estándar (SE) usando Bootstrap**
El bucle utiliza los datos originales de Pandas (que `sklearn` maneja mejor) pero los resultados se consolidan.

In [31]:
N_bootstrap = 1000
bootstrap_coeffs_def = []

print("\nDefault: Cálculo de SE con Bootstrap")
for i in range(N_bootstrap):
    # Remuestreo con reemplazo
    sample_indices = np.random.choice(df_def_encoded.index, size=len(df_def_encoded), replace=True)
    X_boot = X_def_multi.loc[sample_indices]
    y_boot = y_def.loc[sample_indices]
    
    # Ajuste del modelo Logístico (C=1e9 para emular MLE)
    model = LogisticRegression(C=1e9, solver='liblinear').fit(X_boot, y_boot)
    
    # Guardar coeficientes: [Intercepto, balance, income, student]
    bootstrap_coeffs_def.append([model.intercept_[0]] + list(model.coef_[0]))

# Calcular media y desviación estándar (SE Bootstrap)
coeffs_def_array = np.array(bootstrap_coeffs_def)
mean_bootstrap_def = pd.Series(coeffs_def_array.mean(axis=0), index=def_cols).rename('Media Bootstrap')
se_bootstrap_def = pd.Series(coeffs_def_array.std(axis=0), index=def_cols).rename('SE Bootstrap')

print("Resultados de los Coeficientes (Comparación Clásico vs. Bootstrap):")
print(pd.concat([results_def_classic, mean_bootstrap_def, se_bootstrap_def], axis=1))


Default: Cálculo de SE con Bootstrap
Resultados de los Coeficientes (Comparación Clásico vs. Bootstrap):
            Coeficiente Clásico  SE Clásico  Media Bootstrap  SE Bootstrap
Intercepto           -10.869045    0.492273        -0.084746      0.429824
balance                0.005737    0.000232         0.000500      0.000437
income                 0.000003    0.000008        -0.000125      0.000005
student               -0.646776    0.236257        -0.087564      0.396351


# **Regularización L2 (Ridge) en Advertising**

## **Optimización del Hiperparámetro α (o λ)**

In [33]:
# Rango de alfas a probar
alphas_to_test = np.logspace(-4, 2, 100) 

# Ajuste con RidgeCV
ridge_cv = RidgeCV(alphas=alphas_to_test, scoring='neg_mean_squared_error', cv=KFold(10)).fit(X_adv_multi, y_adv)

alpha_optimo = ridge_cv.alpha_

print("\nAdvertising: Optimización L2 (Ridge)")
print(f"Alpha Óptimo (CV): {alpha_optimo:.6f}")


Advertising: Optimización L2 (Ridge)
Alpha ($lambda$) Óptimo (CV): 100.000000


## **Cálculo de SE usando Bootstrap con Regresión Ridge**

In [35]:
N_bootstrap = 1000
bootstrap_coeffs_ridge = []

print("\nAdvertising: Bootstrap con Regresión Ridge")
for i in range(N_bootstrap):
    # Remuestreo
    sample_adv = df_adv.sample(n=len(df_adv), replace=True)
    
    # Ajuste del modelo Ridge con alpha_optimo
    X_boot = sample_adv[['TV', 'radio', 'newspaper']]
    y_boot = sample_adv['sales']
    
    model_ridge = Ridge(alpha=alpha_optimo).fit(X_boot, y_boot)
    
    # Guardar coeficientes
    bootstrap_coeffs_ridge.append([model_ridge.intercept_] + list(model_ridge.coef_))

# Calcular desviación estándar (SE Bootstrap Ridge)
coeffs_ridge_array = np.array(bootstrap_coeffs_ridge)
se_bootstrap_ridge = pd.Series(coeffs_ridge_array.std(axis=0), index=adv_cols).rename('SE Bootstrap (Ridge)')

print("\nSE Coeficientes (Comparación OLS vs. Ridge):")
# Reutilizamos los resultados del OLS clásico para la comparación
print(pd.concat([results_adv_classic['SE Clásico'].rename('SE Clásico (OLS)'), se_bootstrap_ridge], axis=1))


Advertising: Bootstrap con Regresión Ridge

SE Coeficientes (Comparación OLS vs. Ridge):
            SE Clásico (OLS)  SE Bootstrap (Ridge)
const               0.311908                   NaN
TV                  0.001395              0.001971
radio               0.008611              0.010585
newspaper           0.005871              0.006338
Intercepto               NaN              0.331816


# **Conclusiones**

## **Modelo de Regresión Lineal (Advertising)**
El Error Estándar (SE) de Bootstrap es consistentemente mayor que el SE Clásico para todos los coeficientes. Esto indica que los supuestos teóricos del Mínimo Cuadrado Ordinario (OLS) –probablemente la homocedasticidad– están violados. El SE Clásico estaba subestimando la verdadera variabilidad de los coeficientes. El SE Bootstrap proporciona una medida más realista y robusta de la incertidumbre.

### **Tabla Advertising - Comparación OLS vs. Bootstrap**

In [36]:
print("Advertising - OLS vs. Bootstrap (SE y Media)")
adv_table = pd.DataFrame({
    'Coeficiente Clásico': results_adv_classic['Coeficiente Clásico'],
    'SE Clásico': results_adv_classic['SE Clásico'],
    'Media Bootstrap': mean_bootstrap_adv,
    'SE Bootstrap': se_bootstrap_adv
})

adv_table = adv_table.rename(index={'const': 'Intercepto'})
print(adv_table)

Advertising - OLS vs. Bootstrap (SE y Media)
            Coeficiente Clásico  SE Clásico  Media Bootstrap  SE Bootstrap
Intercepto                  NaN         NaN         2.957807      0.335080
TV                     0.045765    0.001395         0.045584      0.001910
Intercepto             2.938889    0.311908              NaN           NaN
newspaper             -0.001037    0.005871        -0.001138      0.006589
radio                  0.188530    0.008611         0.189171      0.010684


## **Modelo de Regresión Logística (Default)**
Este modelo presenta una patología severa que afecta la fiabilidad de los resultados Clásicos (MLE). La enorme diferencia en las magnitudes de los coeficientes (por ejemplo, el Intercepto Clásico −10.87 vs. el Bootstrap −0.08) es una señal de Separación Casi Perfecta de Datos . Esto provoca que los estimadores de Máxima Verosimilitud (MLE) tiendan al infinito (o a valores muy extremos), haciendo que el modelo MLE sea inestable e inconfiable. El SE Bootstrap, siendo mayor, también refleja esta alta variabilidad del modelo.

### **Tabla Default - Comparación MLE vs. Bootstrap**

In [37]:
print("\nDefault - MLE vs. Bootstrap (SE y Media)")
def_table = pd.concat([results_def_classic, mean_bootstrap_def.rename('Media Bootstrap'), se_bootstrap_def.rename('SE Bootstrap')], axis=1)
print(def_table)


Default - MLE vs. Bootstrap (SE y Media)
            Coeficiente Clásico  SE Clásico  Media Bootstrap  SE Bootstrap
Intercepto           -10.869045    0.492273        -0.084746      0.429824
balance                0.005737    0.000232         0.000500      0.000437
income                 0.000003    0.000008        -0.000125      0.000005
student               -0.646776    0.236257        -0.087564      0.396351


## **Regularización L2 (Ridge) en Advertising**
El valor de α=100.000000 (una penalización fuerte) indica que la Validación Cruzada encontró beneficio en encoger los coeficientes. La regularización L2 cumplió su objetivoal reducir el Error Estándar (SE) en la mayoría de los coeficientes (`radio`, `newspaper`, `Intercepto`). Una SE más baja significa que las estimaciones de los coeficientes de Ridge son más estables y menos sensibles a los cambios en la muestra que las estimaciones de OLS.

### **Tabla Advertising - Comparación de Estabilidad (OLS SE vs. Ridge SE)**

In [38]:
print("\nAdvertising - Estabilidad de SE (OLS vs. Ridge)")
ridge_comp_table = pd.DataFrame({
    'SE Clásico (OLS)': results_adv_classic['SE Clásico'],
    'SE Bootstrap (Ridge)': se_bootstrap_ridge
})

ridge_comp_table = ridge_comp_table.rename(index={'const': 'Intercepto'})
print(ridge_comp_table)


Advertising - Estabilidad de SE (OLS vs. Ridge)
            SE Clásico (OLS)  SE Bootstrap (Ridge)
Intercepto               NaN              0.331816
TV                  0.001395              0.001971
Intercepto          0.311908                   NaN
newspaper           0.005871              0.006338
radio               0.008611              0.010585
